In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time

In [ ]:
# driver = webdriver.Chrome()
# driver.get("https://www.ewg.org/tapwater/")

In [ ]:
zip_csv = '../Resources/Data/usa_zipcode_of_or-1554j.csv'
zip_df = pd.read_csv(zip_csv)

In [ ]:
zip_df.dropna()

In [28]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [29]:
zip_list = zip_df.zip.to_list()
# del zip_list[-3:]
# for zip_group in chunker(zip_list,5):
#     print(zip_group)

In [18]:
zip_list = zip_df.zip.to_list()
zip_list.index('97002')
zip_list =zip_list[:10]
# zip_list = zip_list[16:]
# zip_list

In [30]:
#Initialize list to hold the scraped data   
contaminant_list = []

In [31]:
# driver = webdriver.Chrome()
# driver.get("https://www.ewg.org/tapwater/")
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/tapwater/")
#/html/body/div[3]/main/figure[1]/table/tbody/tr[2]/td[1]/a
#/html/body/div[3]/main/figure[1]/table/tbody/tr[2]/td[1]/a

In [1]:
for zip_group in chunker(zip_list,8):
    print(zip_group)
    driver.get("https://www.ewg.org/tapwater/")
    time.sleep(10)
    for zipcode in zip_group:
        print(zipcode)
        #try:
        #driver.manage().timeouts().pageLoadTimeout(40, TimeUnit.SECONDS)

        element = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[1]')
        element.clear()
        element.send_keys(zipcode)
        button = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[3]').click()
        time.sleep(5)
        zip_url = driver.current_url
        url = requests.get(zip_url)
        table = pd.read_html(url.text)
        num_utilities = len(table[0])


      
        #Navigate back to the home page after finishing the zip code scrape    
        driver.back()
        time.sleep(10)
        #except:
            #print('Level one error')
            #pass

NameError: name 'chunker' is not defined

In [ ]:
df = pd.DataFrame(contaminant_list)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

df

In [ ]:
driver.quit()

In [ ]:
df.to_csv('../Resources/Data/OR_zips_scrape.csv')

In [ ]:
with open('../Resources/Data/OR_zips_scrape_after_finished.txt', 'w') as output_file:
    for utility in contaminant_list:
        output_file.write(utility + '\n')

In [ ]:
        #Initialize an empty list to contain all of our data
        for utility_vist in range(num_utilities):
            try:
                print(utility_vist)
                utility_vist = str(utility_vist+1)
                XPath = "/html/body/div[3]/main/figure[1]/table/tbody/tr["+utility_vist+"]/td[1]/a"
                selection = driver.find_element(By.XPATH, XPath).click()
                time.sleep(5)
                # Parse the resulting html with soup
                page_source = driver.page_source
                html_soup = soup(page_source, 'html.parser')

                #Get the name of the Water Utility
                Utility = html_soup.find('h1').text
                Utility

                #get the html data we need
                data_box = html_soup.find_all('div', class_='contaminant-name')

                for i in range(len(data_box)):
                    data = data_box[i].find_all('span')
                    data_measure = []
                    d = {
                        'Utility' : Utility,
                        'Contaminant': '', 
                        'Utility Measuremnt':'', 
                        'EWG HEALTH GUIDELINE': '',
                        'Legal Limit':'' 
                    }

                    contaminant_name = data_box[i].find('h3')
                    d['Contaminant'] = contaminant_name

                    for j in range(len(data)):
                        measurement = data[j].text
                        #print(measurement)
                        data_measure.append(measurement)
                        #print(data_measure)

                    try:
                        d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong")
                    try:
                        d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong") 
                    try:
                        d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
                    except ValueError:
                        print("A value error arose")
                    except:
                        print("Something else went wrong") 

                    contaminant_list.append(d)

                    # Append-adds at last
                    # append mode
                    file1 = open("'../Resources/Data/OR_zips_scrape.txt'", "a")  

                    # writing newline character
                    file1.write(d)

                driver.back()
                time.sleep(5)

            except:
                pass